In [6]:
import numpy as np

from src.common.fitness import griewank
from src.gendered_selection.helpers import (calculate_lifetime)
from src.gendered_selection.conf.gendered_selection_config import Config
from src.gendered_selection.faster_fuzzy_logic.generalized_partition_inferrer import GeneralizedInferrer
from src.gendered_selection.faster_fuzzy_logic.parallel_parition_inferrer import ParallelInferrer

In [7]:
gi = GeneralizedInferrer(n_partitions=4)
pi = ParallelInferrer(n_partitions=4)

In [8]:
population_scale = 1
N =  1000 

genomes = np.random.uniform(-population_scale, population_scale, size=(N, 5))
fitness = np.apply_along_axis(griewank, 1, genomes)
gender = (np.random.rand(N) >= .5).astype(int)
age = np.zeros(N)
age += 1

male_indices = np.argwhere(gender == 1).flatten()
female_indices = np.argwhere(gender == 0).flatten()

lifetime = calculate_lifetime(L=Config.L, U = Config.U, fitness=fitness, age=age)
diversity = age[male_indices] / lifetime[male_indices]
population_diversity = diversity.mean()

In [9]:
%%time
female_preferred_age = np.array([gi.infer_partner_age(age=lifetime[i], diversity=population_diversity) for i in male_indices])

CPU times: user 2.69 s, sys: 751 µs, total: 2.69 s
Wall time: 2.69 s


In [10]:
%%time
result_parralel = pi.multiprocessing_preferred_age(male_indices, lifetime, population_diversity)

CPU times: user 11 ms, sys: 70.4 ms, total: 81.4 ms
Wall time: 525 ms


In [11]:
(result_parralel ==  female_preferred_age).all()

True